In [1]:
import os
import sys
import shutil
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import models


In [2]:
dataset_path = "./Test_Images"
batch_size = 16
epochs = 50

In [3]:
train_dir = "./Test_Images/train"
val_dir = "./Test_Images/val"
test_dir = "./Test_Images/test"

# Create directories for training, validation, and test data
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get paths to all image files in the dataset
image_paths = []
class_labels = os.listdir(dataset_path)
for label in class_labels:
    class_path = os.path.join(dataset_path, label)
    if os.path.isdir(class_path):
        image_paths.extend([os.path.join(class_path, fname) for fname in os.listdir(class_path)])

train_paths, temp_paths = train_test_split(image_paths, test_size=0.2, random_state=42) 
val_paths, test_paths = train_test_split(temp_paths, test_size=0.5, random_state=42)

def move_images(image_paths, dest_dir):
    """
    Move images to their respective class directories.
    """
    for img_path in image_paths:
        if os.path.isfile(img_path):
            label = img_path.split('/')[-2]
            class_dest_dir = os.path.join(dest_dir, label)
            os.makedirs(class_dest_dir, exist_ok=True)
            shutil.copy(img_path, class_dest_dir)

# Move images to their respective directories
move_images(train_paths, train_dir)
move_images(val_paths, val_dir)
move_images(test_paths, test_dir)


In [4]:
train_dir = "./Test_Images/train"
val_dir = "./Test_Images/val"
test_dir = "./Test_Images/test"

# Create directories for training, validation, and test data
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get paths to all image files in the dataset
image_paths = []
class_labels = os.listdir(dataset_path)
for label in class_labels:
    class_path = os.path.join(dataset_path, label)
    if os.path.isdir(class_path):
        image_paths.extend([os.path.join(class_path, fname) for fname in os.listdir(class_path)])

train_paths, temp_paths = train_test_split(image_paths, test_size=0.2, random_state=42) 
val_paths, test_paths = train_test_split(temp_paths, test_size=0.5, random_state=42)

def move_images(image_paths, dest_dir):
    """
    Move images to their respective class directories.
    """
    for img_path in image_paths:
        if os.path.isfile(img_path):
            label = img_path.split('/')[-2]
            class_dest_dir = os.path.join(dest_dir, label)
            os.makedirs(class_dest_dir, exist_ok=True)
            shutil.copy(img_path, class_dest_dir)

# Move images to their respective directories
move_images(train_paths, train_dir)
move_images(val_paths, val_dir)
move_images(test_paths, test_dir)


In [5]:
print(len(train_paths))
print(len(val_paths))
print(len(test_paths))

2477
310
310


In [6]:
class SuppressTensorFlowOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stdout = self._original_stdout

with SuppressTensorFlowOutput():
    train_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical'
    )

with SuppressTensorFlowOutput():
    validation_datagen = ImageDataGenerator(rescale=1./255)
    validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical'
    )

with SuppressTensorFlowOutput():
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical'
    )


In [7]:
# Initialize the base model with InceptionV3
base_model = InceptionV3(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 35s 201ms/step - accuracy: 0.9966 - loss: 0.0142 - val_accuracy: 0.7734 - val_loss: 1.9267
Epoch 2/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 1.0000 - loss: 0.0384 - val_accuracy: 0.6429 - val_loss: 2.1614
Epoch 3/50


2024-11-13 14:39:47.361238: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-13 14:39:47.493811: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


172/172 ━━━━━━━━━━━━━━━━━━━━ 34s 198ms/step - accuracy: 0.9991 - loss: 0.0069 - val_accuracy: 0.7637 - val_loss: 1.9230
Epoch 4/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.8571 - val_loss: 2.2427
Epoch 5/50


2024-11-13 14:40:22.017386: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-13 14:40:22.160112: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


172/172 ━━━━━━━━━━━━━━━━━━━━ 34s 199ms/step - accuracy: 0.9995 - loss: 0.0069 - val_accuracy: 0.7617 - val_loss: 2.0772
Epoch 6/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 1.0000 - loss: 0.0065 - val_accuracy: 0.9286 - val_loss: 0.1019
Epoch 7/50


...

2024-11-13 14:52:46.152816: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-13 14:52:46.295129: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


172/172 ━━━━━━━━━━━━━━━━━━━━ 33s 191ms/step - accuracy: 0.9994 - loss: 0.0042 - val_accuracy: 0.7656 - val_loss: 2.3113
Epoch 48/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 5.5144e-04 - val_accuracy: 0.8571 - val_loss: 1.4218
Epoch 49/50


2024-11-13 14:53:19.594270: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-13 14:53:19.763589: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


172/172 ━━━━━━━━━━━━━━━━━━━━ 37s 214ms/step - accuracy: 0.9993 - loss: 0.0041 - val_accuracy: 0.7676 - val_loss: 2.3238
Epoch 50/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 1.0000 - loss: 0.0046 - val_accuracy: 0.8571 - val_loss: 1.5339


2024-11-13 14:53:56.912547: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-13 14:53:57.057803: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [15]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

In [16]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)

52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 173ms/step - accuracy: 0.8168 - loss: 1.9058
